In [ ]:
from nvidia.dali import pipeline_def
import nvidia.dali.fn as fn
import nvidia.dali.types as types
from nvidia.dali.pipeline import DataNode
import numpy as np
from numpy import array, float32
import os

from utils import index_directory

In [ ]:
image_dir = "/volume1/local_only/pictures"
max_batch_size = 16
filenames = index_directory(image_dir, formats=(".png", ".jpeg", ".jpg"))
seed = -1

In [ ]:
import tritonclient.grpc as grpcclient
from dotenv import load_dotenv

MODEL_VERSION = os.getenv("MODEL_VERSION")
TRITON_SERVER_URL = os.getenv("TRITON_SERVER_URL")
FACE_DETECT_MODEL_NAME = os.getenv("FACE_DETECT_MODEL_NAME")

triton_client = grpcclient.InferenceServerClient(url=TRITON_SERVER_URL, verbose=False)
model_metadata = triton_client.get_model_metadata(
    model_name=FACE_DETECT_MODEL_NAME, model_version=MODEL_VERSION
)
model_config = triton_client.get_model_config(
    model_name=FACE_DETECT_MODEL_NAME, model_version=MODEL_VERSION
).config
input_names = [i.name for i in model_config.input]
output_names = [i.name for i in model_config.output]


@pipeline_def
def client_pipeline(filenames):
    raw_image_tensor, _ = fn.readers.file(files=filenames, random_shuffle=False)
    return raw_image_tensor


raw_image_tensor = client_pipeline(filenames, num_threads=4, batch_size=32, device_id=0)
raw_image_tensor.build()
(raw_image_tensor,) = raw_image_tensor.run()
input_image_data_list = [np.expand_dims(np.array(i), axis=0) for i in raw_image_tensor]

```python
class FacenetPipeline:
    def __init__(self, raw_image_tensor):
        self.raw_image_tensor = raw_image_tensor
        self.one_over_255 = 1 / 255.0
        self.shapes = fn.peek_image_shape(self.raw_image_tensor)

    def load_images(self):
        self.image_tensor = fn.decoders.image(
            self.raw_image_tensor, output_type=types.GRAY, device="mixed"
        )

    def color_space_conversion(self):
        self.image_tensor = fn.color_space_conversion(
            self.image_tensor, image_type=types.GRAY, output_type=types.RGB
        )
        self.image_tensor = fn.brightness_contrast(self.image_tensor)

    def maybe_rotate(self):
        if_rotate = self.shapes[0] > self.shapes[1]
        angle = 90.0 * if_rotate
        self.image_tensor = fn.rotate(self.image_tensor, angle=angle)

    def resize_images(self):
        self.image_tensor = fn.resize(
            self.image_tensor,
            resize_longer=736,
            interp_type=types.DALIInterpType.INTERP_LANCZOS3,
        )
        self.image_tensor = fn.crop(
            self.image_tensor,
            crop_w=736,
            crop_h=416,
            crop_pos_x=0.5,
            crop_pos_y=0.5,
            out_of_bounds_policy="pad",
        )

    @pipeline_def
    def facenet_reshape(self):
        self.load_images()
        self.color_space_conversion()
        self.maybe_rotate()
        self.resize_images()
        self.image_tensor = (
            fn.transpose(self.image_tensor, perm=[2, 0, 1]) * self.one_over_255
        )

        return self.image_tensor, self.shapes


facenet_pipeline = FacenetPipeline(raw_image_tensor)
facenet_pipeline = facenet_pipeline.facenet_reshape(
    batch_size=max_batch_size, num_threads=4, device_id=0, seed=seed
)
facenet_pipeline.build()
raw_image_tensors, shapes = facenet_pipeline.run()
```

In [ ]:
results = []
for input_image_data in input_image_data_list:

    inputs = [grpcclient.InferInput(input_names[0], input_image_data.shape, "UINT8")]
    inputs[0].set_data_from_numpy(input_image_data)

    outputs = [
        grpcclient.InferRequestedOutput(output_name, class_count=0)
        for output_name in output_names
    ]
    response = triton_client.infer(FACE_DETECT_MODEL_NAME, inputs, outputs=outputs)

    results.append(response.as_numpy("true_boxes"))
results

```python
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt

%matplotlib inline


def show_images(image_batch, channels_first=True):
    columns = 4
    rows = (max_batch_size + 1) // (columns)
    fig = plt.figure(figsize=(32, (32 // columns) * rows))
    gs = gridspec.GridSpec(rows, columns)
    for j in range(rows * columns):
        plt.subplot(
            gs[j], title="{} face(s)".format(len(eval(results[j].decode("utf-8"))))
        )
        plt.axis("off")
        if channels_first:
            image = np.transpose(image_batch.at(j), [1, 2, 0])
        else:
            image = image_batch.at(j)
        plt.imshow(image)


show_images(raw_image_tensors.as_cpu())
```

```python
from nvidia.dali import pipeline_def
import nvidia.dali.fn as fn
import nvidia.dali.types as types
from nvidia.dali.plugin.triton import autoserialize


class FacenetPipeline:
    def __init__(self):
        self.raw_image_tensor = fn.external_source(name="input_image_data")
        self.one_over_255 = 1 / 255.0
        self.shapes = fn.peek_image_shape(self.raw_image_tensor)

    def load_images(self):
        self.image_tensor = fn.decoders.image(
            self.raw_image_tensor, output_type=types.GRAY, device="mixed"
        )

    def color_space_conversion(self):
        self.image_tensor = fn.color_space_conversion(
            self.image_tensor, image_type=types.GRAY, output_type=types.RGB
        )
        self.image_tensor = fn.brightness_contrast(self.image_tensor)

    def maybe_rotate(self):
        if_rotate = self.shapes[0] > self.shapes[1]
        angle = 90.0 * if_rotate
        self.image_tensor = fn.rotate(self.image_tensor, angle=angle)

    def resize_images(self):
        self.image_tensor = fn.resize(
            self.image_tensor,
            resize_longer=736,
            interp_type=types.DALIInterpType.INTERP_LANCZOS3,
        )
        self.image_tensor = fn.crop(
            self.image_tensor,
            crop_w=736,
            crop_h=416,
            crop_pos_x=0.5,
            crop_pos_y=0.5,
            out_of_bounds_policy="pad",
        )

    @autoserialize
    @pipeline_def(batch_size=32, num_threads=4, device_id=0)
    def facenet_reshape(self):
        self.load_images()
        self.color_space_conversion()
        self.maybe_rotate()
        self.resize_images()
        self.image_tensor = (
            fn.transpose(self.image_tensor, perm=[2, 0, 1]) * self.one_over_255
        )

        return self.image_tensor, self.shapes


facenet_pipeline = FacenetPipeline()
_ = facenet_pipeline.facenet_reshape().serialize(filename="model.dali")
```